## 1. Loading required packages and functions

In [1]:
%matplotlib inline

from model.dicom_utils import *
import numpy as np 
import pandas as pd 
import os
import pydicom
import matplotlib.pyplot as plt
#import assd_func as af
from skimage import morphology
import matplotlib.patches as mpatches
import model.sduc_src as af_Sobel
import time
import random
from scipy.stats import uniform,norm
import time
import multiprocessing as mp
from model.smooth_3D_contour import *

## 2. Loading in the data using th main function from dicom_utils

In [2]:
def read_exported_contours(path, images, slices):
    contour = read_contours(path)
    labels = get_labels(contour, images.shape, slices, rois=None)
    labels = np.where(labels == True, 1, 0)
    return labels

In [ ]:
Sample subject 11

In [ ]:
dicom_dir = "Sample-prostate-Cases/11"
output_dir = "Sample-output"
images_11, slices_11, contours_11, labels_11, dummy_mask_11 = main(dicom_dir, output_dir)

## 3. Finding organs ID:

In [4]:
def find_roi_x(images, labels, i):
    roi_y = []
    for j in range(labels.shape[2]):
        if True in np.unique(labels[..., i][:, :,  j] > 0):
            roi_y.append(j)
    return roi_y

def find_roi_y(images, labels, i):
    roi_x = []
    for j in range(labels.shape[1]):
        if True in np.unique(labels[..., i][:, j,  :] > 0):
            roi_x.append(j)
    return roi_x

def find_roi_z(images, labels, i):
    roi_z = []
    for j in range(images.shape[0]):
        if True in np.unique(labels[..., i][j, ...] > 0):
            roi_z.append(j)
    return roi_z

def find_organ_i(organ, contours):
    contour_table = pd.DataFrame()
    number = []
    name = []
    for i in range(len(contours)):
        number.append(contours[i]["number"])
        name.append(contours[i]["name"])
    contour_table["number"] = number
    contour_table["name"] = name
    #print(contour_table)
    organ_i =  contour_table[contour_table["name"].str.contains(organ)].index.tolist()[0]
    return organ_i

In [ ]:
def roi_info_extraction(organ, contours, images, labels):
    organ_i_p =  find_organ_i(organ, contours)
    roi_x_p = find_roi_x(images, labels, organ_i_p)
    roi_y_p = find_roi_y(images, labels, organ_i_p)
    roi_z_p = find_roi_z(images, labels, organ_i_p)
    return organ_i_p, roi_x_p, roi_y_p, roi_z_p

organ_p = "prostate" 
organ_r = "rectum" # or Rectum
organ_b = "bladder" # or Bladder

# Subject 11:
print("subject 11")
organ_i_p_11, roi_x_p_11, roi_y_p_11, roi_z_p_11 = roi_info_extraction(organ_p, contours_11, images_11, labels_11)
organ_i_r_11, roi_x_r_11, roi_y_r_11, roi_z_r_11 = roi_info_extraction(organ_r, contours_11, images_11, labels_11)
organ_i_b_11, roi_x_b_11, roi_y_b_11, roi_z_b_11 = roi_info_extraction(organ_b, contours_11, images_11, labels_11)

## 4. Extraction:

## Prostate

In [ ]:
start = time.time()
SD_list1 = [1.7, 1.7, 2]

# Prostate
gt_p_11 = labels_11[..., organ_i_p_11].copy()

num_cores=1
pool = mp.Pool(num_cores)

list_of_c = [5, 50, 100, 200, 300]
#list_of_c = [50, 200, 100, 5, 300]

sub1_results1  = []
for c in list_of_c:
    contour = creating_contour(c, SD_list1, organ_i_p_11, labels_11, images_11, roi_x_p_11, roi_y_p_11, roi_z_p_11, "prostate")
    sub1_results1.append(contour)

                   
#sub1_results1 = [p.get() for p in sub1_results1]  
    
print("---------------------------Prostate done-------------------------------------------")

## Rectum

In [ ]:
# Rectum
#list_of_c = [5, 100, 50, 300, 200]
list_of_c = [5, 50, 100, 200, 300]

gt_r_11 = labels_11[..., organ_i_r_11].copy()

SD_list2 = [1.3, 1.3, 3]
sub1_results2  = []
for c in list_of_c:
    contour = creating_contour(c, SD_list2, organ_i_r_11, labels_11, images_11, roi_x_p_11, roi_y_p_11, roi_z_p_11, "rectum")
    sub1_results2.append(contour)

#sub1_results2 = [p.get() for p in sub1_results2]    

    
print("---------------------------rectum done---------------------------------------------")

## Bladder

In [ ]:
# bldder
#list_of_c = [100, 50, 300, 200, 5]

gt_b_11 = labels_11[..., organ_i_b_11].copy()

SD_list3 = [0.7, 0.7, 2]

sub1_results3  = []
for c in list_of_c:
    contour = creating_contour(c, SD_list3, organ_i_b_11, labels_11, images_11, roi_x_p_11, roi_y_p_11, roi_z_p_11, "bladder")
    sub1_results3.append(contour)

#sub1_results3 = [p.get() for p in sub1_results3]    


print("---------------------------bladder done-------------------------------------------")

## 5. Exportation:

In [ ]:
gt_r_11 = labels_11[..., organ_i_r_11].copy()
gt_b_11 = labels_11[..., organ_i_b_11].copy()

## Exportation
prostate_list = [gt_p_11, sub1_results1[0], sub1_results1[1], sub1_results1[2], sub1_results1[3], sub1_results1[4]]
prostate_name = ["truth_prostate", "auto_prostate_1", "auto_prostate_2", "auto_prostate_3", "auto_prostate_4", "auto_prostate_5"]
prostate_color = ["red", "green", "purple", "yellow", "blue", "orange"]
rectum_list = [gt_r_11, sub1_results2[0], sub1_results2[1], sub1_results2[2], sub1_results2[3], sub1_results2[4]]
rectum_name = ["truth_rectum", "auto_rectum_1", "auto_rectum_2", "auto_rectum_3", "auto_rectum_4", "auto_rectum_5"]
rectum_color = ["red", "purple", "yellow", "blue", "orange", "green"]
bladder_list = [gt_b_11, sub1_results3[0], sub1_results3[1], sub1_results3[2], sub1_results3[3], sub1_results3[4]]
bladder_name = ["truth_bladder", "auto_bladder_1", "auto_bladder_2", "auto_bladder_3", "auto_bladder_4", "auto_bladder_5"]
bladder_color = ["red", "green", "purple", "yellow", "blue", "orange"]

#destination_path = "../"
destination_path = "Sample-output"
RTStruct(prostate_list + rectum_list + bladder_list, prostate_name + rectum_name + bladder_name, color=prostate_color + rectum_color + bladder_color, DICOMImageStruct = slices_11, fname=os.path.join(destination_path, 'subject_11_contours_7_3.dcm'))
end = time.time()
print("Running time:" + str(end-start))
pool.close()